In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [ ]:
data = pd.merge(movies, ratings, on='movieId')
data[['userId','rating','movieId','title']].sort_values('userId').to_csv('data.csv', index=False)
files = pd.read_csv('data.csv')

In [ ]:
content = []
with open('data.csv', encoding="utf-8") as fp:
    content = fp.readlines()
content

In [ ]:
data = {}
for line in content[1:561]:
    line = line.strip().split(',')
    
    if not line[0] in data.keys():
        data[line[0]] = {line[3]:line[1]}
    
    else:
        data[line[0]][line[3]] = line[1]
data

In [6]:
def common_movies(user1,user2):
    common = []
    #取出兩位用戶評論過的電影與評分
    users = [user1, user2]
    user1_data=data[user1]
    user2_data=data[user2]

    #找到兩位使用者都評論過的電影，將這些評過的電影抓出來
    for key in user1_data.keys():
        if key in user2_data.keys():
            common.append(key)

    #將兩位用戶評分轉成矩陣
    user1_rating = []
    user2_rating = []
    for key in common:
        user1_rating.append(float(data[user1][key]))
        user2_rating.append(float(data[user2][key]))

    user1_rating = np.array(user1_rating)
    user2_rating = np.array(user2_rating)
    user1_rating = user1_rating.reshape(1, -1)
    user2_rating = user2_rating.reshape(1, -1)
    if user1_rating.shape == (1, 0) or user2_rating.shape == (1, 0):
        return -1
    else:
        cosine_similarities = cosine_similarity(user1_rating, user2_rating)
    
    return round(cosine_similarities[0][0], 2)

In [7]:
users = ['1', '2', '3', '4', '5']
sim_table = []

for user in users:
    for _user in users:
        sim_table.append(common_movies(user, _user))

sim_table = np.array(sim_table).reshape(5, -1)

print(pd.DataFrame(sim_table, columns=users, index=users))

      1    2     3     4     5
1  1.00  1.0  0.79  0.93  0.97
2  1.00  1.0 -1.00  1.00  1.00
3  0.79 -1.0  1.00  1.00  1.00
4  0.93  1.0  1.00  1.00  0.90
5  0.97  1.0  1.00  0.90  1.00


In [8]:
#取得最相似於某用戶的其他用戶
def top_similar(userID):
    init = sim_table[int(userID) - 1][int(userID) - 1]
    res = {}
    for user in users:
        if not user == userID:
            res[user] = init - sim_table[int(userID) - 1][int(user) - 1]
    return min(res, key=res.get)
top_similar('1')


'2'

In [9]:
def recommend(user):
    #相似度最高的用戶
    top_sim_user = top_similar(user)
    #相似度最高的用戶的觀賞紀錄
    items = data[top_sim_user]
    recommendations = []
    #篩選出該用戶未觀看的電影並加到列表中
    for item in items.keys():
        if item not in data[user].keys():
            recommendations.append((item,items[item]))
    recommendations.sort(key=lambda val:val[1],reverse=True)#按照評分排序
    #回傳評分最高的10部電影
    return recommendations[:10]

col = ['Movie_name', 'Rating']
Recommendations = recommend('1')
re = pd.DataFrame(Recommendations, columns=col, index=[num for num in range(1, 11)])
print(re)

                                           Movie_name Rating
1   The Jinx: The Life and Deaths of Robert Durst ...    5.0
2                                   Inside Job (2010)    5.0
3                                      Warrior (2011)    5.0
4                           Mad Max: Fury Road (2015)    5.0
5                                "Wolf of Wall Street    5.0
6                                Step Brothers (2008)    5.0
7                                               "Town    4.5
8                            Good Will Hunting (1997)    4.5
9                         Inglourious Basterds (2009)    4.5
10                                       "Dark Knight    4.5
